<a href="https://colab.research.google.com/github/Supritha04/WebScrapping/blob/main/reddit_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installing praw and openai

In [ ]:
!pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 7.1 MB/s eta 0:00:00


In [ ]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
import praw

In [ ]:
import openai

### Logging into reddit and openai

In [ ]:
client_id = open("/content/client_id.txt").read()
client_service = open("/content/client_service.txt").read()
password =open("/content/password.txt").read()
user_agent = open("/content/user_agent.txt").read()
username = open("/content/username.txt").read()

In [ ]:
reddit = praw.Reddit(
    client_id=client_id,
    client_secret= client_service,
    password = password,
    user_agent = user_agent,
    username = username,
    check_for_async=False
)

In [ ]:
reddit.user.me()

Redditor(name='Sad_Dust_1026')

In [ ]:
subreddit=reddit.subreddit("python")

In [ ]:
openai.api_key = open("/content/openaikey.txt").read()

### Function to retrieve the comments from url or id

In [ ]:
def get_post_title_and_comments(url):
  submission = reddit.submission(url=url)
  submission.comments.replace_more(limit=None)
  post_title = submission.title
  comments = []
  count = 0
  for comment in submission.comments:
    if count >= 10:
      break
    comments.append(comment.body)
    count += 1
  return post_title, '\n\n'.join(comments)

In [ ]:
# Checking the usage:
url = 'https://www.reddit.com/r/unitedstatesofindia/comments/1b7t9lg/as_a_first_time_voter_who_do_you_plan_to_vote_for/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button'
post_title, comments = get_post_title_and_comments(url)
print("Post Title:", post_title)
print("Comments:", comments)

Post Title: As a first time voter, who do you plan to vote for?
Comments: * If your image submission is Non-OC, provide a link to the source below this comment. 

* If your image submission is OC, mark it as OC, or use [OC]/ (OC) in title, or mention so below this comment.

* Note: Screenshot posts are not allowed. Memes should use proper flair. Links to YouTube channels/ blogs/ websites are only allowed under this comment, do not spam elsewhere.

*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/unitedstatesofindia) if you have any questions or concerns.*

Ppl downvoting each other’s parties in the comments 😭😭😭 theres a war going on here

![img](emote|t5_2kz7we|30378)

Nice try EC!

Doesn't matter who you vote for, that's the sad reality. But if you do wanna make some contribution, forget national picture and focus on local

Ideology Jaan ne ka tareeka thoda kezual hain

I'm from Kerala. Its only a choic

### Function to prepare prompt

In [ ]:
def generate_prompt(post_title, comments):
  prompt = f"### Post Title:\n\n{post_title}\n\n"
  prompt += "### Comments:\n\n"
  prompt += comments + "\n\n"
  prompt += "### Your Comment:\n\n"
  return prompt

In [ ]:
# Checking the usage:
url = 'https://www.reddit.com/r/unitedstatesofindia/comments/1b7t9lg/as_a_first_time_voter_who_do_you_plan_to_vote_for/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button'
post_title, comments = get_post_title_and_comments(url)
prompt = generate_prompt(post_title, comments)
print(prompt)

### Post Title:

As a first time voter, who do you plan to vote for?

### Comments:

* If your image submission is Non-OC, provide a link to the source below this comment. 

* If your image submission is OC, mark it as OC, or use [OC]/ (OC) in title, or mention so below this comment.

* Note: Screenshot posts are not allowed. Memes should use proper flair. Links to YouTube channels/ blogs/ websites are only allowed under this comment, do not spam elsewhere.

*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/unitedstatesofindia) if you have any questions or concerns.*

Ppl downvoting each other’s parties in the comments 😭😭😭 theres a war going on here

![img](emote|t5_2kz7we|30378)

Nice try EC!

Doesn't matter who you vote for, that's the sad reality. But if you do wanna make some contribution, forget national picture and focus on local

Ideology Jaan ne ka tareeka thoda kezual hain

I'm from Kerala. Its o

### Function to generate comment

In [ ]:
def generate_answer(prompt):
  try:
    # Use GPT-3 to generate an answer based on the prompt
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
    {"role": "user", "content": prompt}
    ]
    )
    answer = response['choices'][0]['message']['content'].strip()
    return answer
  except Exception as e:
    print(f"An error occurred: {e}")
    return None

In [ ]:
# Checking the usage
url = 'https://www.reddit.com/r/unitedstatesofindia/comments/1b7t9lg/as_a_first_time_voter_who_do_you_plan_to_vote_for/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button'
post_title, comments = get_post_title_and_comments(url)
prompt = generate_prompt(post_title, comments)
generated_comment=generate_answer(prompt)

In [ ]:
print(generated_comment)

As a first time voter, it's important to educate yourself about the different political parties, their ideologies, and the candidates running in your area. Take the time to research and understand the issues that are important to you before making a decision on who to vote for. Remember, your vote has the power to make a difference, so make it count!


### Function to post the comment

In [ ]:
def post_comment(url, comment_text):
  submission = reddit.submission(url=url)
  submission.reply(comment_text)
  print("Comment posted successfully.")

### The final bot

In [ ]:
def bot(url):
  post_title, comments = get_post_title_and_comments(url)
  prompt = generate_prompt(post_title, comments)
  generated_comment=generate_answer(prompt)
  post_comment(url,generated_comment)

In [ ]:
url="https://www.reddit.com/r/unitedstatesofindia/comments/1b7t9lg/as_a_first_time_voter_who_do_you_plan_to_vote_for/?utm_source=share&utm_medium=web3x&utm_name=web3xcss&utm_term=1&utm_content=share_button"
bot(url)

Comment posted successfully.
